## **Chatbot con `Llama2-7b`**
Construimos un assitente farmacéutico a modo de chatbot, con el modelo `Llama 2-7b`de `Meta`

---
### **Librerías**

In [1]:
import json
import numpy as np
import torch
import sys
import os
from sentence_transformers import SentenceTransformer

import matplotlib.pyplot as plt

# utils
# Agrega la ruta del directorio donde está el utils al path de Python
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from utils import load_json

# from sklearn.decomposition import uPCA
import seaborn as sns
import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GPT2Tokenizer, GPT2LMHeadModel

/opt/miniconda3/envs/PharmAI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
### **1. Cargar el modelo**

---
##### **`Llama2-7b` (normal o chat)**

In [2]:
def load_llama_model():

    # Detectar el dispositivo disponible: CUDA, MPS (para Mac con Apple Silicon) o CPU
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    print("Usando dispositivo:", device)

    # Nombre del modelo a cargar (Llama-2-7b)
    #model_name = "meta-llama/Llama-2-7b-hf" # Llama2 normal
    model_name = "meta-llama/Llama-2-7b-chat-hf" # Llama2 chat
    
    # Cargar el tokenizador
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

    # Cargar el modelo, especificando el tipo de datos y usando device_map="auto" para aprovechar la GPU
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device in ["cuda", "mps"] else torch.float32,
        device_map="auto",
    )

    return model, tokenizer

# Cargar el modelo
model, tokenizer = load_llama_model()

Usando dispositivo: mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.12s/it]
Some parameters are on the meta device because they were offloaded to the disk.


---
##### **`GPT2`**

In [3]:
def load_gpt2_model():
    # Detectar el dispositivo: MPS para Mac con Apple Silicon o CPU
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    print("Usando dispositivo:", device)

    model_name = "gpt2"  # Puedes cambiar a "gpt2-medium" o "gpt2-large" si lo deseas
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Mover el modelo al dispositivo seleccionado
    model.to(device)
    return model, tokenizer

# Cargar el modelo GPT-2
#model, tokenizer = load_gpt2_model()

---
##### **`GPT-J-6b`**

In [4]:
def load_gpt_j_model():
    # Detectar el dispositivo disponible: CUDA, MPS (para Mac con Apple Silicon) o CPU
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    print("Usando dispositivo:", device)

    # Nombre del modelo a cargar (GPT-J-6B)
    model_name = "EleutherAI/gpt-j-6B"

    # Cargar el tokenizador
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

    # Cargar el modelo, especificando el tipo de datos y usando device_map="auto" para aprovechar la GPU
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device in ["cuda", "mps"] else torch.float32,
        device_map="auto",
    )

    return model, tokenizer

# Cargar el modelo
#model, tokenizer = load_gpt_j_model()

---
### **2. Funciones para buscar el contexto en la BBDD vectorial**

In [5]:
def retrieve_relevant_fragments_prueba(query, model, fragments, index, k=5):
    """
    Recupera los fragmentos más relevantes para la consulta utilizando un modelo de similitud (ej. FAISS).

    Parámetros:
    - query (str): Consulta del usuario.
    - top_k (int): Número de fragmentos a recuperar.

    Retorna:
    - list: Lista de fragmentos relevantes.
    """
    # Simulación de la búsqueda, usando FAISS o similar.
    # Esto debería ser reemplazado por la implementación real que recupera los fragmentos relevantes.
    # Suponemos que "retrieved_fragments" es el resultado de una búsqueda en base de datos vectorial.
    '''
    retrieved_fragments = [
        {
            "medicamento": "Aspirina",
            "categoria": "efectos_secundarios",
            "texto": "Puede causar náuseas y dolor de estómago.",
        },
        {
            "medicamento": "Paracetamol",
            "categoria": "efectos_secundarios",
            "texto": "Puede causar problemas hepáticos en dosis altas.",
        },
    ]
    '''
    retrieved_fragments = [
        {
            "medicamento": "Paracetamol",
            "categoria": "efectos_secundarios",
            "texto": "Puede causar problemas hepáticos en dosis altas.",
        }
    ]
    return retrieved_fragments

In [6]:
def retrieve_relevant_fragments(query, embedding_model, fragments, index, k=10):
    """
    Realiza una búsqueda en FAISS para encontrar los fragmentos más similares a la consulta.

    Parámetros:
    - query (str): La consulta en lenguaje natural.
    - k (int): Número de resultados a recuperar.

    Retorna:
    - Lista de fragmentos de texto relevantes.
    """
    
    # Convertir la consulta en embedding
    query_embedding = embedding_model.encode(query, convert_to_numpy=True).reshape(1, -1)

    # Buscar los k embeddings más cercanos
    distances, indices = index.search(query_embedding, k)

    # Recuperar los fragmentos correspondientes, incluyendo las distancias
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(fragments):  # Asegurar que el índice es válido
            results.append(
                {
                    **fragments[idx],  # Añadir los datos del fragmento
                    "distance": distances[0][i],  # Añadir la distancia de similitud
                }
            )

    return results

In [7]:
def format_context(retrieved_fragments, max_fragments=5, max_text_length=2000):
    """
    Formatea los fragmentos recuperados en un contexto para el modelo. Transforma una lista de diccionarios en un texto estructurado.

    Parámetros:
    - retrieved_fragments (list): Lista de fragmentos recuperados (diccionarios)
    - max_fragments (int): Número máximo de fragmentos a utilizar
    - max_text_length (int): Longitud máxima del texto a mostrar por fragmento

    Retorna:
    - str: Contexto formateado para el modelo
    """
    context = ""
    
    # Asegurar que no se intenten tomar más fragmentos de los que existen
    num_fragments = min(len(retrieved_fragments), max_fragments)

    for i, frag in enumerate(retrieved_fragments[:num_fragments]):
        # Verificar que cada fragmento tenga las claves necesarias
        if not all(key in frag for key in ["medicamento", "categoria", "texto"]):
            print(f"Advertencia: Fragmento {i+1} no tiene la estructura esperada.")
            continue  # Saltar fragmentos mal formateados

        medicamento = frag["medicamento"]
        categoria = frag["categoria"]
        texto = frag["texto"]

        # Limitar la longitud del texto
        truncated_text = (
            texto[:max_text_length] + "..." if len(texto) > max_text_length else texto
        )

        # Construcción del contexto
        context += f"\nFragmento {i+1}:\n"
        context += f"Medicamento: {medicamento}\n"
        context += f"Categoría: {categoria}\n"
        context += f"Información: {truncated_text}\n"

    return context

---
### **3. Función para construir el Prompt**

##### **Prompt resumido**

In [8]:
'''
def build_prompt(context, query):
    """
    Construye el prompt para el modelo con base en el contexto y la consulta,
    incluyendo un ejemplo de cómo debe formatear la respuesta.

    Parámetros:
    - context (str): Contexto a proporcionar al modelo
    - query (str): Consulta del usuario

    Retorna:
    - str: Prompt completo para el modelo
    """
    prompt = f"""Eres un asistente médico especializado en información sobre medicamentos.
    Basándote ÚNICAMENTE en la siguiente información sobre medicamentos:

    {context}

    Responde de manera clara y precisa a esta pregunta: {query}

    Ejemplo de consulta:
    Pregunta: ¿Cuáles son los efectos secundarios de la aspirina?
    Contexto: 
    Fragmento 1:
    Medicamento: Aspirina
    Categoría: reacciones_adversas
    Información: Los efectos secundarios comunes incluyen náuseas, dolor de estómago y sangrados.

    Respuesta:
    La aspirina puede causar efectos secundarios como náuseas, dolor de estómago y sangrados. Si necesitas más detalles, por favor consulta la ficha técnica completa.

    Si la información proporcionada no es suficiente para responder completamente, indica qué datos faltan, pero no inventes información ni dejes la respuesta vacía.

    Tu respuesta debe ser:
    1. Precisa y basada solo en el contexto proporcionado.
    2. Estructurada y fácil de entender.
    3. Sin añadir información que no esté en los fragmentos.
    4. Con referencias claras al medicamento mencionado.

    Respuesta:"""
    return prompt
'''


'\ndef build_prompt(context, query):\n    """\n    Construye el prompt para el modelo con base en el contexto y la consulta,\n    incluyendo un ejemplo de cómo debe formatear la respuesta.\n\n    Parámetros:\n    - context (str): Contexto a proporcionar al modelo\n    - query (str): Consulta del usuario\n\n    Retorna:\n    - str: Prompt completo para el modelo\n    """\n    prompt = f"""Eres un asistente médico especializado en información sobre medicamentos.\n    Basándote ÚNICAMENTE en la siguiente información sobre medicamentos:\n\n    {context}\n\n    Responde de manera clara y precisa a esta pregunta: {query}\n\n    Ejemplo de consulta:\n    Pregunta: ¿Cuáles son los efectos secundarios de la aspirina?\n    Contexto: \n    Fragmento 1:\n    Medicamento: Aspirina\n    Categoría: reacciones_adversas\n    Información: Los efectos secundarios comunes incluyen náuseas, dolor de estómago y sangrados.\n\n    Respuesta:\n    La aspirina puede causar efectos secundarios como náuseas, dolo

##### **Prompt mejorado**

In [9]:
def build_prompt(context, query):
    """
    Construye el prompt para el modelo con base en el contexto y la consulta,
    incluyendo un ejemplo de cómo debe formatear la respuesta.

    Parámetros:
    - context (str): Contexto a proporcionar al modelo
    - query (str): Consulta del usuario

    Retorna:
    - str: Prompt completo para el modelo
    """
    prompt = f"""
    
    1. OBJETIVO GENERAL:
    Eres un asistente médico especializado en información sobre medicamentos. Debes responder a la pregunta del usuario basándote únicamente en la información proporcionada. No debes inventar ni suponer información adicional.

    2. FORMATO DEL CONTEXTO:
    El contexto se presenta como un texto con varios fragmentos que contiene información de uno o varios medicamentos presentes en la pregunta del ususario, donde cada fragmento tiene el siguiente formato:
    - Medicamento: Nombre del medicamento
    - Categoría: Categoría de la información (ej. efectos secundarios, interacciones)
    - Información: Texto relevante sobre el medicamento, el cual debes analizar antes de responder.

    3. FORMATO DE RESPUESTA:
    - Debes responder de manera clara y precisa a la pregunta formulada por el usuario, utilizando ÚNICAMENTE el contexto que se te está proporcionando. 
    - Si la información proporcionada no es suficiente para responder completamente, indica qué datos faltan.
    - Incluye una mención explícita a los textos que respaldan tu respuesta, indicando para todos ellos el medicamento y la categoría.

    4. EJEMPLOS DE CONSULTA Y DE RESPUESTA:
    - EJEMPLO 1:
        Pregunta: ¿Cuáles son los efectos secundarios de la aspirina?
        Contexto:
            "medicamento": "Aspirina"
            "categoria": "efectos_secundarios"
            "texto": "Puede causar náuseas y dolor de estómago."
        Respuesta:
        La aspirina puede causar efectos secundarios como náuseas y dolor de estómago (extraído de la sección "efectos_secundarios" del medicamento "ASPIRINA": "la aspirina tiene como efectos secundarios, entre ottros, la aparición de náuseas y dolor de tripa"). Si necesitas más detalles, por favor consulta la ficha técnica completa.

    - EJEMPLO 2:
        Pregunta: ¿Puedo tomar medicamentoA si estoy embarazada?
        Contexto:
            "medicamento": "medicamentoA"
            "categoria": "contraindicaciones"
            "texto": "No se recomienda su uso durante el embarazo."
        Respuesta:
        No se recomienda el uso de medicamentoA durante el embarazo (extraído de la sección "fertilidad_embarazo" del medicamento "medicamentoA": "el uso de medicamentoA durante el embarazo puede entrañar riesgos, por lo que se desaconseja totalmente su uso en embarazadas"). Si necesitas más detalles, por favor consulta la ficha técnica completa.

    5. INSTRUCCIONES FINALES:
    Básandote ÚNICAMENTE en la información proporcionada en ({context}), responde a la siguiente pregunta:
    {query}

    6. RECUERDA:
    - No debes inventar información ni suponer datos que no estén presentes en el contexto.
    - Si es posible, referencia el fragmento específico que respalda tu respuesta, indicando el medicamento y la categoría.
    - Si la información proporcionada no es suficiente para responder completamente, indica qué datos faltan.
    - Si la pregunta no está relacionada con medicamentos, indica que no puedes ayudar en ese caso.

    Respuesta:"""
    return prompt

---
### **4. Función para generar respuestas basadas en el contexto y el prompt**

In [10]:
def generate_answer(query, context, tokenizer, model):
    """
    Genera una respuesta basada en los fragmentos recuperados usando LLaMA.

    Parámetros:
    - query (str): La consulta del usuario
    - context (string): Texto formateado con los fragmentos recuperados 
    - tokenizer: Tokenizador del modelo
    - model: Modelo generativo

    Retorna:
    - str: Respuesta generada
    """

    # Construimos el prompt para el modelo
    prompt = build_prompt(context, query)

    # Tokenizamos el prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Detectar el dispositivo disponible: CUDA, MPS (para Mac con Apple Silicon) o CPU
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    print("Usando dispositivo:", device)

    input_ids = input_ids.to(device)

    # Generamos la respuesta usando el modelo
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + 2000,  # Limita la longitud de salida
            do_sample=True,
            temperature=0.1,
            top_p=0.9,
            repetition_penalty=1.2,
        )

    # Decodificamos la respuesta generada
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extraemos solo la parte de la respuesta después del prompt
    response = response[
        len(prompt) :
    ].strip()  # Ajuste para capturar la respuesta correctamente

    return response

---
### **5. Función para realizar la consulta**

In [ ]:
def answer_query(query, model, tokenizer):
    """
    Realiza una consulta y genera una respuesta utilizando el modelo.

    Parámetros:
    - query (str): La consulta del usuario

    Retorna:
    - str: Respuesta generada
    """

    # 1. Converir la consulta a minúsculas
    query = query.lower()

    # 2. Recuperamos los fragmentos relevantes para la consulta
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # old
    fragments = load_json("../../data/outputs/5_chatbot/contexto_medicamentos_chatbot.json") 
    index = faiss.read_index("../../data/outputs/5_chatbot/faiss_index_all-MiniLM-L6-v2.bin") # old

    # 3. Busca los fragmentos relevantes
    #retrieved_fragments = retrieve_relevant_fragments_prueba(query, embedding_model, fragments, index, k=5)
    retrieved_fragments = retrieve_relevant_fragments(query, embedding_model, fragments, index, k=5)

    # 4. Aplicamos formateo al contexto
    print(f"Fragmentos recuperados: {retrieved_fragments}")
    context = format_context(retrieved_fragments)

    # 5. Generamos la respuesta del modelo
    print(f"Contexto: {context}")
    response = generate_answer(query, context, tokenizer, model)

    return response

---
### **6. Ejemplo de Consulta**

In [ ]:
#query = "¿Cuáles son las reacciones adversas del paracetamol?"
query = "¿Cuáles son los efectos a la hora de conducir del ibuprofeno?" # Funciona bien
query = "¿Cuáles son las reacciones adversas del ibuprofeno?"response = answer_query(query, model, tokenizer)
print("Respuesta generada:", response)

Fragmentos recuperados: [{'medicamento': 'PARACETAMOL_MABO_1_g_COMPRIMIDOS_EFG', 'categoria': 'contraindicaciones', 'texto': 'hipersensibilidad al paracetamol, o a alguno de los excipientes.', 'distance': np.float32(0.4822652)}, {'medicamento': 'ANTIDOL_NOCHE_500_MG_25_MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA', 'categoria': 'contraindicaciones', 'texto': '- - hipersensibilidad al paracetamol, a difenhidramina o a alguno de los excipientes incluidos en la sección 6.1. porfiria.', 'distance': np.float32(0.5046341)}, {'medicamento': 'DOLOMIDINA_500_25MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA', 'categoria': 'contraindicaciones', 'texto': '- - hipersensibilidad al paracetamol, a difenhidramina o a alguno de los excipientes incluidos en la sección 6.1. porfiria.', 'distance': np.float32(0.5046341)}, {'medicamento': 'DOLOSTOP_500_MG_COMPRIMIDOS', 'categoria': 'contraindicaciones', 'texto': 'hipersensibilidad al paracetamol o a alguno de los excipientes.', 'distance': np.float32(0.505387)}, {'med